In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [2]:
L63_data_path = '../data/L63-trajectories'
save_folder='../data/rf-start'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(1e4)
save_interval = 100
N = 20000
L0 = 0.4
L1 = 3.5
beta = 4e-5
partition = [100, 100, 100]
train = np.load(f'{L63_data_path}/train.npy').astype(np.float32)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(np.float32)

model = srnn.SurrogateModel_NN(3, 300, name='nn', save_folder=save_folder)
model.init_with_rf(L0, L1, beta, train, partition)
# model.learn(train[:, :N], steps, learning_rate, beta, log_interval, save_interval, milestones, drop, batch_size=N-1)

In [3]:
iters = list(range(0, 10000, 100))
# tau = np.zeros(len(iters))
# for i, iter in enumerate(iters):
#     model.net = torch.load(f'{model.save_folder}/nn_{iter}')
#     tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
#     tau[i] = tau_f_se.mean()
#     print(i)

In [4]:
# plt.plot(iters, tau, label="test_model")
# plt.xlabel('iteration (descent step)')
# plt.ylabel(r'mean $\tau_f$')
# plt.legend()
# plt.savefig(f'{model.save_folder}/iter_vs_mean_tau_f.png')

In [5]:
model.init_with_rf(L0, L1, beta, train, partition=[200, 50, 50])
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)

Time taken by compute_tau_f is 1.4002 seconds


In [6]:
tau_f_se

array([4.8048, 6.461 , 5.642 , 6.2062, 4.4408, 4.1496, 9.7916, 8.1718,
       6.9888, 6.4246, 3.1304, 6.006 , 6.1698, 5.4782, 6.1516, 7.3346,
       3.8038, 6.2972, 4.3498, 6.825 , 3.8766, 4.7684, 7.4984, 5.0778,
       8.9726, 6.097 , 7.462 , 5.6056, 4.2224, 5.1324, 5.8968, 6.4064,
       2.548 , 3.003 , 8.281 , 5.7148, 7.644 , 3.7128, 5.1142, 5.9696,
       6.8614, 7.644 , 6.0788, 5.8058, 7.6804, 6.1516, 7.1708, 3.1304,
       2.3478, 8.0808, 8.0808, 2.5844, 4.4408, 7.5712, 5.4782, 4.9504,
       5.3326, 6.6066, 5.1324, 4.5136, 5.2416, 4.8958, 5.4964, 5.0232,
       5.1688, 4.368 , 5.642 , 4.9322, 5.6966, 5.7876, 4.9504, 6.8068,
       5.4782, 4.641 , 6.6066, 4.9868, 6.1516, 4.4772, 5.46  , 7.28  ,
       6.6976, 8.4084, 7.5166, 6.1334, 6.7704, 8.1172, 3.7128, 4.5318,
       6.8796, 5.4782, 4.9322, 6.734 , 5.46  , 8.0262, 6.1698, 6.0424,
       5.4236, 5.3326, 5.8422, 6.4246])

In [7]:
model.learn(train[:, :N], 10, learning_rate, beta, log_interval, save_interval, milestones, drop, batch_size=5)
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)

step: 0    loss: 2.664762     time elapsed=0.0230
Time taken by learn is 0.3224 seconds
Time taken by compute_tau_f is 1.3464 seconds


In [8]:
tau_f_se

array([0.    , 0.0182, 0.    , 0.    , 0.0182, 0.    , 0.0182, 0.    ,
       0.0182, 0.    , 0.0182, 0.    , 0.    , 0.    , 0.0182, 0.    ,
       0.0182, 0.    , 0.0182, 0.0182, 0.0182, 0.    , 0.    , 0.0182,
       0.0182, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.0182,
       0.0182, 0.    , 0.0182, 0.0182, 0.    , 0.0182, 0.    , 0.0182,
       0.    , 0.0182, 0.    , 0.    , 0.    , 0.    , 0.0182, 0.0182,
       0.    , 0.    , 0.    , 0.0182, 0.0182, 0.0182, 0.    , 0.    ,
       0.    , 0.    , 0.0182, 0.0182, 0.0182, 0.0182, 0.    , 0.0182,
       0.0182, 0.0182, 0.0182, 0.0182, 0.    , 0.0182, 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.0182, 0.0182, 0.0182, 0.    , 0.    ,
       0.    , 0.    , 0.0182, 0.    , 0.0182, 0.0182, 0.0182, 0.0182,
       0.    , 0.0182, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    ])

In [9]:
model.net(next(model.dataloader.__iter__())[0])

tensor([[ -9.2252,  -3.4536,  44.8248],
        [-11.8170, -19.2823,  31.4022],
        [  2.4676,   2.5145,  23.6541],
        [  7.8561,  10.0131,  23.5841],
        [  6.3446,  10.8706,  13.8709]], grad_fn=<MmBackward0>)

In [10]:
train[:N]

array([[ -8.361066 ,  -9.38598  , -10.454209 , ...,  -5.5973125,
         -4.3846292,  -3.3684328],
       [-13.313976 , -14.654672 , -15.823077 , ...,   0.951339 ,
          1.1862783,   1.2309328],
       [ 18.146687 ,  19.625534 ,  21.55841  , ...,  31.42136  ,
         29.684444 ,  28.051142 ]], dtype=float32)